<a href="https://colab.research.google.com/github/radamp11/nlp_words_correction/blob/develop/nlp_words_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torchvision.models as models

import datetime

In [ ]:
print('hello world')

hello world


In [ ]:
# # if we need e.g. saving to spreadsheets on google drive
# from google.colab import auth
# auth.authenticate_user()

# # if we need to connect our google drive for e.g. load text corpora
# from google.colab import drive
# drive.mount('/content/drive')

TOOLS INSTALLATION

In [ ]:
# installing essentiall tools via !pip install -U
!pip install -U spacy
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 6.2 MB 5.2 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 457 kB 68.0 MB/s 
     |████████████████████████████████| 660 kB 57.9 MB/s 
     |████████████████████████████████| 181 kB 68.7 MB/s 
     |████████████████████████████████| 10.1 MB 53.2 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    U

VOCABULARY AND DATASET CLASS


In [ ]:
# reads words from corpora and embeds them into tokens using sentencepiece and collects them in vocabulary dict
class Vocabulary:
    pass

In [ ]:
# uses Vocabulary class; returns item via __getitem__
class WordsDataset(Dataset):
    def __init__(self, root_dir, text_corpora_file, freq_threshold=5):
        pass

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        pass

In [ ]:
# optional - may be not necessary
# collate is needed for batch processing to fill the len difference between sentences with <PAD> token to make them the same length 
class Collate:
    pass

MODEL

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, dropout):
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.num_layers = num_layers
        self.hidden_size = hidden_size

    def forward(self, input): 
        # TODO
        input = self.dropout(input) # embed?
        hiddens,  = self.lstm(input)
        outputs = self.linear(hiddens)
        return outputs


PARAMETERS

In [ ]:
embed_size = 1024 #?
hidden_size = 1024
num_layers = 2
learning_rate = 0.001
num_epochs = 20
batch_size = 32
dropout = 0.4
vocab_size = 0 # len of dataset

checkpoint_filename = (f"date:{str(datetime.datetime.now())}"
                       f"_embed:{str(embed_size)}"
                       f"_hidden:{str(hidden_size)}"
                       f"_lay:{str(num_layers)}"
                       f"_lr:{str(learning_rate)}"
                       f"_epochs:{str(num_epochs)}"
                       f"_batch:{str(batch_size)}"
                       f"_drop:{str(dropout)}"
                       f".pth.tar")

LOADERS

In [ ]:
def get_loader(root_dir, text_corpora_file, freq_threshold):
    loader = DataLoader()
    dataset = WordsDataset(root_dir, text_corpora_file, freq_threshold)
    return loader, dataset

In [ ]:
train_loader, dataset = get_loader()

TypeError: ignored

TRAINING

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers, dropout).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.train()
start = datetime.datetime.now()
print("Training initialized at: ", start)
for epoch in range(num_epochs):

    for idx, (stuff_from_getitem_method1, stuff2) in enumerate(train_loader):
        with torch.autograd.set_detect_anomaly(True):
            stuff_from_getitem_method1 = stuff_from_getitem_method1.to(device)
            stuff2 = stuff2.to(device)
            optimizer.zero_grad()

            outputs, _ = model(stuff_from_getitem_method1, stuff2)

            # loss = criterion(outputs.reshape(-1, outputs.shape[2]), reference_text_with_no_errors[1:].reshape(-1))
            # loss.backward(loss)
            optimizer.step()   

    # saving to checkpoint step
    if epoch % 10 == 0 and epoch != 0:
        torch.save([model, optimizer], 'path_to_your_google_drive' + f'epoch{str(epoch)}_' + checkpoint_filename)

    # validate(model, device, val_loader, dataset, f'add_att_1024_1layer_fixed_loss/add_att_1024_1layer_epoch{str(epoch)}_loss{str(round(loss.item(),6))}_' + result_worksheet_name, vis_att=True)
    model.train()
        
    # print("Epoch no. ", epoch, ", timestamp: ", datetime.datetime.now())
    # print("Loss: {:.6f}...".format(loss.item()))

end = datetime.datetime.now()
print(f"The training took {end - start} seconds")

GENERATING CONFUSION SET

In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np
import spacy as spacy
import regex as re
import json


def levenshtein_distance(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2

    a = 0
    b = 0
    c = 0

    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if token1[t1 - 1] == token2[t2 - 1]:
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]

                if a <= b and a <= c:
                    distances[t1][t2] = a + 1
                elif b <= a and b <= c:
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return int(distances[len(token1)][len(token2)])

input_path = 'drive/MyDrive/nlp_projekt/news.2021.en.shuffled.deduped'
output_path = 'drive/MyDrive/nlp_projekt/levensthein' + str(sentence_limit) + '_confusion_set.json'
input_example = 'example.txt'
output_example = 'example_out.json'

sentence_limit = 10000

spacy_en = spacy.load('en_core_web_lg')
english_word_regex = r"([A-Z]|[a-z]|')"


def max_distance_criterium(word1, word2):
    max_distance = 3
    if len(word1) <= 2 or len(word2) <= 2:
        max_distance = 2
    return max_distance


start = datetime.now()


def create_set(input_file, output_file, sentence_limit):
    # dict containing word as a key, and confusing words as a value
    new_confusion_dict = {}
    unique_words = set()
    with open(input_file, encoding="utf-8") as f:
        print("Reading " + str(sentence_limit) + " sentences from a file...")
        for i in range(sentence_limit):
            # reading all the file at once may end in memory error
            line = f.readline()
            if not line:
                break
            line = line[:-2]    # remove \n
            list_of_words = [tok.text for tok in spacy_en.tokenizer(line)]
            for word in list_of_words:
                if re.match(english_word_regex, word):
                    unique_words.add(word)

    unique_words_list = list(unique_words)
    print("Create dictionary. Words to process: " + str(len(unique_words_list)))
    start_in = datetime.now()
    for i in range(len(unique_words_list) - 1):
        if i % 1000 == 0 and i != 0:
            print("already processed " + str(i) + " words in " + str(datetime.now() - start_in) + ' time...')
            start_in = datetime.now()
        confusion_list = []
        if len(unique_words_list[i]) < 2:
            continue
        for j in range(len(unique_words_list) - 1):
            if len(unique_words_list[j]) < 2 or abs(len(unique_words_list[i]) - len(unique_words_list[j])) > 2:
                continue
            if i != j and levenshtein_distance(unique_words_list[i], unique_words_list[j]) < \
                    max_distance_criterium(unique_words_list[i], unique_words_list[j]):
                confusion_list.append(unique_words_list[j])
        new_confusion_dict[unique_words_list[i]] = confusion_list

    with open(output_file, 'w', encoding='utf-8') as output:
        output.write(json.dumps(new_confusion_dict))


create_set(input_path, output_path, sentence_limit)
# create_set(input_example, output_example, sentence_limit)
print("Whole process took: " + str(datetime.now() - start))


Reading 10000 sentences from a file...
Create dictionary. Words to process: 29256
